In [ ]:
# IO packages
from aicsimageio import AICSImage
import os
import imageio

# calculation packages
import numpy as np

# segmentation packages
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from aicssegmentation.core.seg_dot import dot_3d_wrapper
from skimage.morphology import dilation, ball, remove_small_objects

# watershed packages
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy.ndimage import distance_transform_edt
from skimage.measure import label



In [ ]:
def segment_RNA(image_name, filepath, output_path):
    """ Takes a filepath that is a single channel tif as an input
    Segments the image - optimized for RNA
    Returns a segmented image
    """
        
    print('Segmenting RNA for ' + filepath)

    
    reader = AICSImage(filepath) 
    IMG = reader.data

    #####################
    structure_channel = 0
    #####################

    structure_img0 = IMG[0, 0, structure_channel,:,:,:]
    ################################
    # First, calculate the best intensity normalization parameters
    
    minimum_value = np.amin(structure_img0)
    mean_value = np.mean(structure_img0)
    percentile_99 = np.percentile(structure_img0, 99.99)
    std_array = np.std(structure_img0)

    a = round((mean_value - minimum_value) / std_array, 1)
    b = round((percentile_99 - mean_value) / std_array, 1)
    
    ################################
    ## PARAMETER ##
    intensity_scaling_param = [a, b]
    gaussian_smoothing_sigma = 0.5
    ################################

    # intensity normalization
    structure_img = intensity_normalization(structure_img0, scaling_param=intensity_scaling_param)

    # smoothing with gaussian filter
    structure_img_smooth = image_smoothing_gaussian_slice_by_slice(structure_img, sigma=gaussian_smoothing_sigma)
    
    ################################
    ## Detect spots w/ 3D dot wrapper ##
    
    s3_param = [[1, 0.005], [10, 0.1]]
    bw = dot_3d_wrapper(structure_img_smooth, s3_param)

    #################################
    # Watershed to resolve close together particles
   
    minArea = 20
    local_maxi = peak_local_max(structure_img0,labels=label(bw), min_distance=5, indices=False)
    distance = distance_transform_edt(bw)
    im_watershed = watershed(-distance, label(dilation(local_maxi, selem=ball(1))), mask=bw, watershed_line=True)
  
    ################################
    ## PARAMETERS for removing small objects ##
    minArea = 20
    ################################

    final_seg = remove_small_objects(im_watershed>0, min_size=minArea, connectivity=1, in_place=False)
    
    ################################
    ## SAVE TO DISK
    ################################
    
    output_seg = final_seg>0
    out=output_seg.astype(np.uint8)
    out[out>0]=255
    imageio.volwrite(output_path + image_name, out)

        
    return None

In [ ]:

rna_raw_data_dir = '/data/rna/raw-data/'

output_path = '/data/rna/segmentations/'


if not os.path.isdir(output_path):
    os.makedirs(output_path)


image_ls = os.listdir(rna_raw_data_dir)

for image_name in image_ls:
    if not image_name[0] == '.':
        image_path = os.path.join(rna_raw_data_dir, image_name)

        segment_RNA(image_name, image_path, output_path)
